In [1]:
import numpy as np
import pandas as pd

import keras
from keras.models import load_model
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_recall_curve, auc, classification_report

## Start Here

In [2]:
filePath = './draft/data_train_valid.npz'
data = np.load(filePath)

X_train = data['X_train']
y_train = data['y_train']
X_valid = data['X_valid']
y_valid = data['y_valid']

In [13]:
class EnsembleClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
        self.model_lst = load_model(models[0])
        self.model_gru = load_model(models[1])
        self.model_tcn = load_model(models[2])
        self.ttl_weight = sum(weights)
        self.lst_weight = weights[0] / self.ttl_weight
        self.gru_weight = weights[1] / self.ttl_weight
        self.tcn_weight = weights[2] / self.ttl_weight

    def fit(self, X, y):
        return self

    def predict(self, X):
        lst_pred = self.model_lst.predict(X)
        gru_pred = self.model_gru.predict(X)
        tcn_pred = self.model_tcn.predict(X)
        ensemble_pred = (lst_pred * self.lst_weight + gru_pred * self.gru_weight + tcn_pred * self.tcn_weight)
        return (ensemble_pred > 0.5).astype(int)

    def predict_proba(self, X):
        lst_pred = self.model_lst.predict(X)
        gru_pred = self.model_gru.predict(X)
        tcn_pred = self.model_tcn.predict(X)
        ensemble_pred = (lst_pred * self.lst_weight + gru_pred * self.gru_weight + tcn_pred * self.tcn_weight)
        return ensemble_pred

In [14]:
def ensembleModel(models: list, weights: list):
    ensemble_clf = EnsembleClassifier(models=models, weights=weights)

    # # Training Data
    # # Classification Report
    # y_train_pred = ensemble_clf.predict(X_train)
    # y_train_prob = ensemble_clf.predict_proba(X_train)
    # print("Classification Report for Training Data:")
    # print(classification_report(y_train, y_train_pred))
    
    # # ROC-AUC Score
    # roc_auc_train = roc_auc_score(y_train, y_train_prob)
    # print(f'ROC AUC Score (Training): {roc_auc_train}')
    
    # # Precision-Recall Curve and AUC
    # precision_train, recall_train, _ = precision_recall_curve(y_train, y_train_prob)
    # pr_auc_train = auc(recall_train, precision_train)
    # print(f'Precision-Recall AUC (Training): {pr_auc_train}')

    # # Validation Data
    y_valid_pred = ensemble_clf.predict(X_valid)
    y_valid_prob = ensemble_clf.predict_proba(X_valid)

    # Classification Report
    print("\nClassification Report for Validation Data:")
    print(classification_report(y_valid, y_valid_pred))
    
    # ROC-AUC Score
    roc_auc_valid = roc_auc_score(y_valid, y_valid_prob)
    print(f'ROC AUC Score (Validation): {roc_auc_valid}')
    
    # Precision-Recall Curve and AUC
    precision_valid, recall_valid, _ = precision_recall_curve(y_valid, y_valid_prob)
    pr_auc_valid = auc(recall_valid, precision_valid)
    print(f'Precision-Recall AUC (Validation): {pr_auc_valid}')

#### LSTM : GRU : TCN

## Percobaan Utama

### Manual pada Epoch 10

#### Dengan Bobot 1:1:1

In [5]:
# 0.90 : 0.88 : 0.93
ensembleModel(models=['./rev_model/3_LSTM_lr_1e-4.keras', 
                      './rev_model/7_GRU_lr_1e-4.keras', 
                      './rev_model/12_TCN_lr_auto.keras'], 
               weights=[1, 1, 1]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9586739668491712
Precision-Recall AUC (Validation): 0.9546064432006812


#### Dengan Bobot 1:1:2

In [6]:
# 0.90 : 0.88 : 0.93
ensembleModel(models=['./rev_model/3_LSTM_lr_1e-4.keras', 
                      './rev_model/7_GRU_lr_1e-4.keras', 
                      './rev_model/12_TCN_lr_auto.keras'], 
               weights=[1, 1, 2]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       201
           1       0.94      0.88      0.91       199

    accuracy                           0.91       400
   macro avg       0.91      0.91      0.91       400
weighted avg       0.91      0.91      0.91       400

ROC AUC Score (Validation): 0.9569989249731243
Precision-Recall AUC (Validation): 0.9538534600070361


### Manual pada Epoch 20

#### Dengan Bobot 1:1:1

In [7]:
# 0.92 : 0.94 : 0.92
ensembleModel(models=['./rev_model/15_LSTM_lr_1e-4.keras', 
                      './rev_model/19_GRU_lr_1e-4.keras', 
                      './rev_model/24_TCN_lr_auto.keras'], 
               weights=[1, 1, 1]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9575239380984526
Precision-Recall AUC (Validation): 0.9482617238088886


#### Dengan Bobot 1:2:1

In [8]:
# 0.92 : 0.94 : 0.92
ensembleModel(models=['./rev_model/15_LSTM_lr_1e-4.keras', 
                      './rev_model/19_GRU_lr_1e-4.keras', 
                      './rev_model/24_TCN_lr_auto.keras'], 
               weights=[1, 2, 1]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.958023950598765
Precision-Recall AUC (Validation): 0.9488506575852521


### Manual pada Epoch 30

#### Dengan Bobot 1:1:1

In [9]:
# 0.92 : 0.93 : 0.92
ensembleModel(models=['./rev_model/27_LSTM_lr_1e-4.keras', 
                      './rev_model/31_GRU_lr_1e-4.keras', 
                      './rev_model/36_TCN_lr_auto.keras'], 
               weights=[1, 1, 1]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.90      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9665741643541088
Precision-Recall AUC (Validation): 0.9619200154177225


#### Dengan Bobot 1:2:1

In [10]:
# 0.92 : 0.93 : 0.92
ensembleModel(models=['./rev_model/27_LSTM_lr_1e-4.keras', 
                      './rev_model/31_GRU_lr_1e-4.keras', 
                      './rev_model/36_TCN_lr_auto.keras'], 
               weights=[1, 2, 1]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.90      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9669241731043275
Precision-Recall AUC (Validation): 0.9621028452307283


### Callback EarlyStopping

#### Dengan Bobot 1:1:1

In [11]:
# 0.91 : 0.93 : 0.92
ensembleModel(models=['./rev_model/39_LSTM_lr_1e-4.keras', 
                      './rev_model/43_GRU_lr_1e-4.keras', 
                      './rev_model/47_TCN_lr_1e-4.keras'], 
               weights=[1, 1, 1]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9635990899772494
Precision-Recall AUC (Validation): 0.9569198575979111


#### Dengan Bobot 1:2:1

In [12]:
# 0.91 : 0.93 : 0.92
ensembleModel(models=['./rev_model/39_LSTM_lr_1e-4.keras', 
                      './rev_model/43_GRU_lr_1e-4.keras', 
                      './rev_model/47_TCN_lr_1e-4.keras'], 
               weights=[1, 2, 1]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9637240931023275
Precision-Recall AUC (Validation): 0.954481813295584


## Percobaan Masukan

Menggunakan nilai performa model dasar/base learner yang didapatkan dari hasil perhitungan dua (2) metrik evaluasi yaitu akurasi dan f1-score yang telah dipaparkan sebelumnya.

### Manual pada Epoch 10

In [15]:
# 0.90 : 0.88 : 0.93
ensembleModel(models=['./rev_model/3_LSTM_lr_1e-4.keras', 
                      './rev_model/7_GRU_lr_1e-4.keras', 
                      './rev_model/12_TCN_lr_auto.keras'], 
               weights=[0.90, 0.88, 0.93]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9585989649741244
Precision-Recall AUC (Validation): 0.9546120501026013


### Manual pada Epoch 20

In [16]:
# 0.92 : 0.94 : 0.92
ensembleModel(models=['./rev_model/15_LSTM_lr_1e-4.keras', 
                      './rev_model/19_GRU_lr_1e-4.keras', 
                      './rev_model/24_TCN_lr_auto.keras'], 
               weights=[0.92, 0.94, 0.92]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.957548938723468
Precision-Recall AUC (Validation): 0.9482691625575319


### Manual pada Epoch 30

In [17]:
# 0.92 : 0.93 : 0.92
ensembleModel(models=['./rev_model/27_LSTM_lr_1e-4.keras', 
                      './rev_model/31_GRU_lr_1e-4.keras', 
                      './rev_model/36_TCN_lr_auto.keras'], 
               weights=[0.92, 0.93, 0.92]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 156ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.90      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9665991649791243
Precision-Recall AUC (Validation): 0.9620848697129614


### Callback EarlyStopping

In [18]:
# 0.91 : 0.93 : 0.92
ensembleModel(models=['./rev_model/39_LSTM_lr_1e-4.keras', 
                      './rev_model/43_GRU_lr_1e-4.keras', 
                      './rev_model/47_TCN_lr_1e-4.keras'], 
               weights=[0.91, 0.93, 0.92]
             )

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.89      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

ROC AUC Score (Validation): 0.9636240906022651
Precision-Recall AUC (Validation): 0.956941359721098
